INSTALACIÓN DE LIBRERIAS

In [53]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu110 (from versions: 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121)
ERROR: No matching distribution found for torch==1.7.1+cu110

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


IMPORTACIÓN DE LIBRERIAS

In [56]:
import pyodbc
import torch
from scipy.spatial.distance import cosine
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz
from unidecode import unidecode
from datetime import datetime
from pymongo import MongoClient
import re
import pyodbc
import numpy as np

Conexión BDD RELACIONAL

In [57]:
# Establece la cadena de conexión a tu servidor SQL Server
server = 'BFICHASIST23'
database = 'laboratorio'
username = 'sa'
password = '1719895086'

conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TIMEOUT=30')

In [58]:
# Ejecutamos el query para realizar la consulta a la base
base_med = "SELECT id_medico, apellido_paterno, apellido_materno, nombres, numero_identificacion, id_especialidad FROM dbo.medicos"

#  Se guarda en un df 
sql_data = pd.read_sql_query(base_med, conn)

#Se guarda el df en una nueva versión para no afectar la primera tabla
sql_data2 = sql_data

#Se concatena el nombre y apellidos para poder ejecutar el modelo
sql_data2['NOMBRE_COMPLETO_SQL'] = sql_data['apellido_paterno'] + ' ' + sql_data['apellido_materno'] + ' ' + sql_data['nombres']

# Filtrar las columnas con las que se va a trabajar
sql_data2 = sql_data2.iloc[:,[0,6,5,4]]

# Visualización del df
sql_data2


C:\Users\jquispe\AppData\Local\Temp\ipykernel_8068\1258988748.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_data = pd.read_sql_query(base_med, conn)


,id_medico,NOMBRE_COMPLETO_SQL,id_especialidad,numero_identificacion
0,1,ROMERO HERRERA DENISE ELCIRA,6,0920116530
1,2,FLORES YANANGOMEZ DIANA ALEXANDRA,11,1103532808
2,3,NAVEDA PEREZ DIANA MARICELA,4,1804125001
3,4,COELLO AGUILAR DIANA MARLENE,15,0702216052
4,5,JUAREZ CARPIO DIEGO EDMUNDO,2,0701072472
...,...,...,...,...
3105,3106,RODRIGUEZ CEVALLOS JESSICA JASMIN,30,0703585331
3106,3107,ESCOBAR MURILLO JESSICA ELIZABETH,21,0603229337
3107,3108,GUEVARA SEVILLANO JESSICA CECIBEL,42,0802056796
3108,3109,GUILLEN GARCIA JESSICA ASUNCION,32,1309208393


In [59]:
# Ejecutamos el query para realizar la consulta a la base
especialidad = "select id_especialidad ID_ESPECIALIDAD, nombre_especialidad ESPECIALIDAD from dbo.especialidades"
# Se guarda en un df 
especialidad = pd.read_sql_query(especialidad, conn)
especialidad.head()

C:\Users\jquispe\AppData\Local\Temp\ipykernel_8068\804103328.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  especialidad = pd.read_sql_query(especialidad, conn)


,ID_ESPECIALIDAD,ESPECIALIDAD
0,1,ALERGOLOGIA
1,2,CARDIOLOGIA
2,3,CIRUGIA CARDIOVASCULAR
3,4,CIRUGIA GENERAL
4,5,CIRUGIA ORAL Y MAXILOFACIAL


CARGA DE DATOS MEDICOS NUEVOS

In [60]:
# Lee el archivo Excel en un DataFrame
med_nuevos = pd.read_excel('C:/Users/jquispe/Desktop/JACQUE/MAESTRÍA/tesis/NLP/INSUMO/ALTAS MED.xlsm', skiprows=1)
#Para no afectar la tabla inicial generamos una copia
med_nuevos2 = med_nuevos
#Seleccionamos las columnas con las que se va a trabajar
med_nuevos2 = med_nuevos2.iloc[:, :-7].copy()
#Se concatena el nombre y apellidos para poder ejecutar el modelo
med_nuevos2['NOMBRE_COMPLETO_EXC'] = med_nuevos2['APELLIDO_PAT'] + ' ' + med_nuevos2['APELLIDO_MAT'] + ' ' + med_nuevos2['NOMBRES']
med_nuevos2.head()

c:\Users\jquispe\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,ESPECIALIDAD,DIRECCION,NOMBRE_COMPLETO_EXC
0,2581015,ARCE,CAMPOVERDE,RAFAEL MARCOS,702916859,1985-02-23,MARCOS.ARCE@GMAIL.COM,998756321,2,PEDIATRÍA,AVENIDA 3 OE 9874 Y CALLE 1,ARCE CAMPOVERDE RAFAEL MARCOS
1,2581017,CARDENAS,DIAZ,FAUSTO OLIVERIO,1400237044,1960-10-31,FAUSTO.CARDENAS@GMAIL.COM,963144522,4,TRAUMATOLOGÍA Y ORTOPEDIA,CALLE 4 E 1452 Y CALLE 8,CARDENAS DIAZ FAUSTO OLIVERIO
2,2581019,FRIEDMAN,MATELUNA,DANIEL FERNANDO,1708790819,1993-12-18,DANIEL.FRIEDMAN@GMAIL.COM,968521452,2,MEDICINA GENERAL,CALLE 6 N 52156 Y CALLE 11,FRIEDMAN MATELUNA DANIEL FERNANDO
3,2581021,MATAMOROS,ORELLANA,KAREN LIZETTE,704012392,1966-05-24,KAREN.MATAMOROS@GMAIL.COM,995555111,4,MEDICINA GENERAL,AVENIDA 7 N 9876 Y AVENIDA 65,MATAMOROS ORELLANA KAREN LIZETTE
4,2581023,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,1981-10-09,MARIA.MIRANDA@GMAIL.COM,963124451,2,MEDICINA GENERAL,AVENIDA 11 N 6546 Y CALLE 5,MIRANDA GARCES MARIA DE LOURDES


In [64]:
# Realiza la fusión (merge) de los DataFrames para obtener el código de especialidad
med_nuevos2 = med_nuevos2.merge(especialidad, on='ESPECIALIDAD', how='left')
# Renombra la columna 'Codigo' a 'Especialidad'
med_nuevos2 = med_nuevos2.rename(columns={'id_especialidad': 'cod_Espe'})
# Eliminar una columna específica del DataFrame
med_nuevos2 = med_nuevos2.drop('ESPECIALIDAD', axis=1)
med_nuevos2

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD
0,2581015,ARCE,CAMPOVERDE,RAFAEL MARCOS,0702916859,1985/02/23,MARCOS.ARCE@GMAIL.COM,0998756321,2,AVENIDA 3 OE 9874 Y CALLE 1,ARCE CAMPOVERDE RAFAEL MARCOS,37
1,2581017,CARDENAS,DIAZ,FAUSTO OLIVERIO,1400237044,1960/10/31,FAUSTO.CARDENAS@GMAIL.COM,0963144522,4,CALLE 4 E 1452 Y CALLE 8,CARDENAS DIAZ FAUSTO OLIVERIO,42
2,2581019,FRIEDMAN,MATELUNA,DANIEL FERNANDO,1708790819,1993/12/18,DANIEL.FRIEDMAN@GMAIL.COM,0968521452,2,CALLE 6 N 52156 Y CALLE 11,FRIEDMAN MATELUNA DANIEL FERNANDO,23
3,2581021,MATAMOROS,ORELLANA,KAREN LIZETTE,0704012392,1966/05/24,KAREN.MATAMOROS@GMAIL.COM,0995555111,4,AVENIDA 7 N 9876 Y AVENIDA 65,MATAMOROS ORELLANA KAREN LIZETTE,23
4,2581023,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,1981/10/09,MARIA.MIRANDA@GMAIL.COM,0963124451,2,AVENIDA 11 N 6546 Y CALLE 5,MIRANDA GARCES MARIA DE LOURDES,23
5,2581025,PILLAJO,BALLADARES,GLORIA JANETH,1712510419,1973/12/06,GLORIA.PILLAJO@GMAIL.COM,0991125244,4,AVENIDA 16 OE 546 Y CALLE 852,PILLAJO BALLADARES GLORIA JANETH,23
6,2581024,PEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992/09/18,MARTHA.P@EZ@GMAIL.COM,0987444112,3,AVENIDA 14 S 85541,PEZ SANCHEZ MARTHA CECILIA,12
7,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995/02/14,RITA.BRAVO@GMAIL.COM,0985214566,3,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL,15
8,2581028,MORALES,PEREZ,PIEDAD EMMA,0501600373,1962/11/01,EMMAPIEDAD.MORALESPEREZ@HOTMAIL.COM,0987126890,2,CALLE 4842 NRO 5655 Y CALLE 3866,MORALES PEREZ PIEDAD EMMA,5
9,2581030,CASTILLO,MORAN,IVETTE CECIBEL,0908333602,1990/12/17,IVETTECECIBEL.MORANCASTILLO@HOTMAIL.COM,0977440964,3,CALLE 4819 NRO 6809 Y CALLE 2730,CASTILLO MORAN IVETTE CECIBEL,28


In [61]:
#Visualización Tipo de Datos de las variables del archivo cargado
med_nuevos2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CODCUP               11 non-null     int64         
 1   APELLIDO_PAT         11 non-null     object        
 2   APELLIDO_MAT         11 non-null     object        
 3   NOMBRES              11 non-null     object        
 4   CI                   11 non-null     int64         
 5   FEC_NAC              11 non-null     datetime64[ns]
 6   EMAIL                11 non-null     object        
 7   TEL                  11 non-null     int64         
 8   Q_FICO               11 non-null     int64         
 9   ESPECIALIDAD         11 non-null     object        
 10  DIRECCION            11 non-null     object        
 11  NOMBRE_COMPLETO_EXC  11 non-null     object        
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 1.2+ KB


TRATAMIENTO DE DATOS

In [62]:
# Función para eliminar tildes y convertir "ñ" en "n" en una cadena
def quitar_tildes_y_n(cadena):
    if isinstance(cadena, str):
        cadena_sin_tildes = unidecode(cadena)
        cadena_sin_n = cadena_sin_tildes.replace('ñ', 'n')
        return cadena_sin_n
    else:
        return cadena
    
# Aplicar la función eliminar tildes y ñ solo a las columnas de tipo objeto (texto)
for columna in med_nuevos2.select_dtypes(include=['object']).columns:
    med_nuevos2[columna] = med_nuevos2[columna].apply(quitar_tildes_y_n) # esta funcion quita las tildes y las ñ

# Eliminación de espacios en blanco
med_nuevos2[columna] = med_nuevos2[columna].str.strip()

# elimina las filas duplicadas 
med_nuevos2 = med_nuevos2.drop_duplicates() 

# Convierte en tipo texto
med_nuevos2['TEL'] = med_nuevos2['TEL'].apply(lambda x: str(x).replace('.0', ''))
med_nuevos2['Q_FICO'] = med_nuevos2['Q_FICO'].apply(lambda x: str(x).replace('.0', ''))
med_nuevos2['CI'] = med_nuevos2['CI'].astype(str)

# Función para que la cedula y celular tenga la estructura de 10 digitos
def convertir_a_texto(valor):
    if isinstance(valor, str):
        valor_texto = str(valor)
        if len(valor_texto) == 9:
            valor_texto = '0' + valor_texto
        return valor_texto
    else:
        return str(valor)
    
# Aplicar la función solo a las columnas "ci" y "celular"
med_nuevos2['TEL'] = med_nuevos2['TEL'].apply(convertir_a_texto)
med_nuevos2['CI'] = med_nuevos2['CI'].apply(convertir_a_texto)
    

# Función para eliminar caracteres especiales de una cadena
def eliminar_caracteres_especiales(cadena):
    if isinstance(cadena, str):
        return re.sub(r'[^\w\s]', '', cadena)
    else:
        return cadena

# Obtener las columnas de texto (excepto la columna de correo)
columnas_texto = [col for col in med_nuevos2.select_dtypes(include=['object']).columns if col != 'EMAIL']

# Iterar a través de las columnas de texto y aplicar la función
for columna in columnas_texto:
    med_nuevos2[columna] = med_nuevos2[columna].apply(eliminar_caracteres_especiales)

# convierte en mayusculas absolutamente todo el texto 
med_nuevos2 = med_nuevos2.applymap(lambda x: x.upper() if isinstance(x, str) else x) 


# Visualizar los cambios
med_nuevos2

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,ESPECIALIDAD,DIRECCION,NOMBRE_COMPLETO_EXC
0,2581015,ARCE,CAMPOVERDE,RAFAEL MARCOS,0702916859,1985-02-23,MARCOS.ARCE@GMAIL.COM,0998756321,2,PEDIATRIA,AVENIDA 3 OE 9874 Y CALLE 1,ARCE CAMPOVERDE RAFAEL MARCOS
1,2581017,CARDENAS,DIAZ,FAUSTO OLIVERIO,1400237044,1960-10-31,FAUSTO.CARDENAS@GMAIL.COM,0963144522,4,TRAUMATOLOGIA Y ORTOPEDIA,CALLE 4 E 1452 Y CALLE 8,CARDENAS DIAZ FAUSTO OLIVERIO
2,2581019,FRIEDMAN,MATELUNA,DANIEL FERNANDO,1708790819,1993-12-18,DANIEL.FRIEDMAN@GMAIL.COM,0968521452,2,MEDICINA GENERAL,CALLE 6 N 52156 Y CALLE 11,FRIEDMAN MATELUNA DANIEL FERNANDO
3,2581021,MATAMOROS,ORELLANA,KAREN LIZETTE,0704012392,1966-05-24,KAREN.MATAMOROS@GMAIL.COM,0995555111,4,MEDICINA GENERAL,AVENIDA 7 N 9876 Y AVENIDA 65,MATAMOROS ORELLANA KAREN LIZETTE
4,2581023,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,1981-10-09,MARIA.MIRANDA@GMAIL.COM,0963124451,2,MEDICINA GENERAL,AVENIDA 11 N 6546 Y CALLE 5,MIRANDA GARCES MARIA DE LOURDES
5,2581025,PILLAJO,BALLADARES,GLORIA JANETH,1712510419,1973-12-06,GLORIA.PILLAJO@GMAIL.COM,0991125244,4,MEDICINA GENERAL,AVENIDA 16 OE 546 Y CALLE 852,PILLAJO BALLADARES GLORIA JANETH
6,2581024,PEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992-09-18,MARTHA.P@EZ@GMAIL.COM,0987444112,3,GASTROENTEROLOGIA,AVENIDA 14 S 85541,PEZ SANCHEZ MARTHA CECILIA
8,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995-02-14,RITA.BRAVO@GMAIL.COM,0985214566,3,GINECOLOGIA Y OBSTETRICIA,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL
9,2581028,MORALES,PEREZ,PIEDAD EMMA,0501600373,1962-11-01,EMMAPIEDAD.MORALESPEREZ@HOTMAIL.COM,0987126890,2,CIRUGIA ORAL Y MAXILOFACIAL,CALLE 4842 NRO 5655 Y CALLE 3866,MORALES PEREZ PIEDAD EMMA
10,2581030,CASTILLO,MORAN,IVETTE CECIBEL,0908333602,1990-12-17,IVETTECECIBEL.MORANCASTILLO@HOTMAIL.COM,0977440964,3,NEUMOLOGIA,CALLE 4819 NRO 6809 Y CALLE 2730,CASTILLO MORAN IVETTE CECIBEL


In [63]:
#Fecha de nacimiento a texto y con el formato adecuado para cargar a las bases
med_nuevos2['FEC_NAC'] = med_nuevos2['FEC_NAC'].dt.strftime('%Y/%m/%d')

med_nuevos2


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,ESPECIALIDAD,DIRECCION,NOMBRE_COMPLETO_EXC
0,2581015,ARCE,CAMPOVERDE,RAFAEL MARCOS,0702916859,1985/02/23,MARCOS.ARCE@GMAIL.COM,0998756321,2,PEDIATRIA,AVENIDA 3 OE 9874 Y CALLE 1,ARCE CAMPOVERDE RAFAEL MARCOS
1,2581017,CARDENAS,DIAZ,FAUSTO OLIVERIO,1400237044,1960/10/31,FAUSTO.CARDENAS@GMAIL.COM,0963144522,4,TRAUMATOLOGIA Y ORTOPEDIA,CALLE 4 E 1452 Y CALLE 8,CARDENAS DIAZ FAUSTO OLIVERIO
2,2581019,FRIEDMAN,MATELUNA,DANIEL FERNANDO,1708790819,1993/12/18,DANIEL.FRIEDMAN@GMAIL.COM,0968521452,2,MEDICINA GENERAL,CALLE 6 N 52156 Y CALLE 11,FRIEDMAN MATELUNA DANIEL FERNANDO
3,2581021,MATAMOROS,ORELLANA,KAREN LIZETTE,0704012392,1966/05/24,KAREN.MATAMOROS@GMAIL.COM,0995555111,4,MEDICINA GENERAL,AVENIDA 7 N 9876 Y AVENIDA 65,MATAMOROS ORELLANA KAREN LIZETTE
4,2581023,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,1981/10/09,MARIA.MIRANDA@GMAIL.COM,0963124451,2,MEDICINA GENERAL,AVENIDA 11 N 6546 Y CALLE 5,MIRANDA GARCES MARIA DE LOURDES
5,2581025,PILLAJO,BALLADARES,GLORIA JANETH,1712510419,1973/12/06,GLORIA.PILLAJO@GMAIL.COM,0991125244,4,MEDICINA GENERAL,AVENIDA 16 OE 546 Y CALLE 852,PILLAJO BALLADARES GLORIA JANETH
6,2581024,PEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992/09/18,MARTHA.P@EZ@GMAIL.COM,0987444112,3,GASTROENTEROLOGIA,AVENIDA 14 S 85541,PEZ SANCHEZ MARTHA CECILIA
8,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995/02/14,RITA.BRAVO@GMAIL.COM,0985214566,3,GINECOLOGIA Y OBSTETRICIA,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL
9,2581028,MORALES,PEREZ,PIEDAD EMMA,0501600373,1962/11/01,EMMAPIEDAD.MORALESPEREZ@HOTMAIL.COM,0987126890,2,CIRUGIA ORAL Y MAXILOFACIAL,CALLE 4842 NRO 5655 Y CALLE 3866,MORALES PEREZ PIEDAD EMMA
10,2581030,CASTILLO,MORAN,IVETTE CECIBEL,0908333602,1990/12/17,IVETTECECIBEL.MORANCASTILLO@HOTMAIL.COM,0977440964,3,NEUMOLOGIA,CALLE 4819 NRO 6809 Y CALLE 2730,CASTILLO MORAN IVETTE CECIBEL


ALGORTIMO VERIFICADOR DE CÉDULAS DE IDENTIDAD

In [65]:
def validar_cedula(cedula):
    # Verificar que la cédula tenga 10 dígitos
    if len(cedula) != 10:
        return False
    
    # Verificar que los primeros dos dígitos sean válidos
    provincia = int(cedula[:2])
    if provincia < 1 or provincia > 24:
        return False
    
    # Verificar el tercer dígito (dígito de control)
    tercer_digito = int(cedula[2])
    if tercer_digito < 0 or tercer_digito > 5:
        return False
    
    # Verificar el último dígito (dígito de verificación)
    verificacion = int(cedula[9])
    
    # Calcular el dígito de verificación esperado
    coeficientes = [2, 1, 2, 1, 2, 1, 2, 1, 2]
    suma = 0
    for i in range(9):
        digito = int(cedula[i])
        producto = digito * coeficientes[i]
        if producto > 9:
            producto -= 9
        suma += producto
    
    verificacion_esperada = 10 - (suma % 10)
    if verificacion_esperada == 10:
        verificacion_esperada = 0
    
    # Comparar el dígito de verificación calculado con el proporcionado
    return verificacion == verificacion_esperada

# Aplicacion de la funcion
med_nuevos2['verif_ced'] = med_nuevos2['CI'].apply(validar_cedula)
med_nuevos2


,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD,verif_ced
0,2581015,ARCE,CAMPOVERDE,RAFAEL MARCOS,0702916859,1985/02/23,MARCOS.ARCE@GMAIL.COM,0998756321,2,AVENIDA 3 OE 9874 Y CALLE 1,ARCE CAMPOVERDE RAFAEL MARCOS,37,True
1,2581017,CARDENAS,DIAZ,FAUSTO OLIVERIO,1400237044,1960/10/31,FAUSTO.CARDENAS@GMAIL.COM,0963144522,4,CALLE 4 E 1452 Y CALLE 8,CARDENAS DIAZ FAUSTO OLIVERIO,42,True
2,2581019,FRIEDMAN,MATELUNA,DANIEL FERNANDO,1708790819,1993/12/18,DANIEL.FRIEDMAN@GMAIL.COM,0968521452,2,CALLE 6 N 52156 Y CALLE 11,FRIEDMAN MATELUNA DANIEL FERNANDO,23,True
3,2581021,MATAMOROS,ORELLANA,KAREN LIZETTE,0704012392,1966/05/24,KAREN.MATAMOROS@GMAIL.COM,0995555111,4,AVENIDA 7 N 9876 Y AVENIDA 65,MATAMOROS ORELLANA KAREN LIZETTE,23,True
4,2581023,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,1981/10/09,MARIA.MIRANDA@GMAIL.COM,0963124451,2,AVENIDA 11 N 6546 Y CALLE 5,MIRANDA GARCES MARIA DE LOURDES,23,True
5,2581025,PILLAJO,BALLADARES,GLORIA JANETH,1712510419,1973/12/06,GLORIA.PILLAJO@GMAIL.COM,0991125244,4,AVENIDA 16 OE 546 Y CALLE 852,PILLAJO BALLADARES GLORIA JANETH,23,True
6,2581024,PEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992/09/18,MARTHA.P@EZ@GMAIL.COM,0987444112,3,AVENIDA 14 S 85541,PEZ SANCHEZ MARTHA CECILIA,12,True
7,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995/02/14,RITA.BRAVO@GMAIL.COM,0985214566,3,CALLE 3 N 1243 Y CALLE 7,BRAVO QUIJANO RITA ANNABEL,15,True
8,2581028,MORALES,PEREZ,PIEDAD EMMA,0501600373,1962/11/01,EMMAPIEDAD.MORALESPEREZ@HOTMAIL.COM,0987126890,2,CALLE 4842 NRO 5655 Y CALLE 3866,MORALES PEREZ PIEDAD EMMA,5,True
9,2581030,CASTILLO,MORAN,IVETTE CECIBEL,0908333602,1990/12/17,IVETTECECIBEL.MORANCASTILLO@HOTMAIL.COM,0977440964,3,CALLE 4819 NRO 6809 Y CALLE 2730,CASTILLO MORAN IVETTE CECIBEL,28,True


MODELO DE DETECCION DE DUPLICADOS

MODELO FUZZY TOKEN SORT RATIO

In [66]:
# Crear una lista para almacenar las coincidencias
coincidencias = []

# Iterar a través de los nombres en el archivo Excel
for index_excel, row_excel in med_nuevos2.iterrows():
    nombre_excel = row_excel['NOMBRE_COMPLETO_EXC']
    mejor_coincidencia = None
    mejor_porcentaje = 0

    for index_db, row_db in sql_data2.iterrows():
        nombre_db = row_db['NOMBRE_COMPLETO_SQL']

        # Calcular la similitud entre los nombres y apellidos
        similitud_nombre = fuzz.token_sort_ratio(nombre_excel.lower(), nombre_db.lower())

        # Si la similitud está por encima del umbral y es mejor que la anterior, actualizar la mejor coincidencia
        if similitud_nombre > mejor_porcentaje:
            mejor_coincidencia = pd.concat([row_excel, row_db])
            mejor_coincidencia['Porcentaje de Similitud'] = similitud_nombre
            mejor_porcentaje = similitud_nombre

    # Agregar la mejor coincidencia a la lista de coincidencias
    if mejor_coincidencia is not None:
        coincidencias.append(mejor_coincidencia)

# Crear un DataFrame con las mejores coincidencias
coincidencias_df = pd.DataFrame(coincidencias)

# Función para asignar "Medico en Base" o "Medico Nuevo" en función de la similitud
def asignar_medico(row):
    return "MEDICO EN BASE" if row['Porcentaje de Similitud'] > 79 else "MEDICO NUEVO"

# Aplicar la función a cada fila y crear una nueva columna "OBSERVACION"
coincidencias_df['OBSERVACION'] = coincidencias_df.apply(asignar_medico, axis=1)

# Validación de cédulas
coincidencias_df['VAL_CEDULA'] = coincidencias_df['CI'] == coincidencias_df['numero_identificacion']

# Validación de especialidad
coincidencias_df['VAL_ESPECIALIDAD'] = coincidencias_df['ID_ESPECIALIDAD'] == coincidencias_df['id_especialidad']

# Visualización de resultados
coincidencias_df

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,...,ID_ESPECIALIDAD,verif_ced,id_medico,NOMBRE_COMPLETO_SQL,id_especialidad,numero_identificacion,Porcentaje de Similitud,OBSERVACION,VAL_CEDULA,VAL_ESPECIALIDAD
0,2581015,ARCE,CAMPOVERDE,RAFAEL MARCOS,0702916859,1985/02/23,MARCOS.ARCE@GMAIL.COM,0998756321,2,AVENIDA 3 OE 9874 Y CALLE 1,...,37,True,259,SARMIENTO CAMPOVERDE MARCO TULIO,15,0101625671,66,MEDICO NUEVO,False,False
1,2581017,CARDENAS,DIAZ,FAUSTO OLIVERIO,1400237044,1960/10/31,FAUSTO.CARDENAS@GMAIL.COM,0963144522,4,CALLE 4 E 1452 Y CALLE 8,...,42,True,2590,CARDENAS VELIZ RUTH ELIZABETH,36,1709324907,62,MEDICO NUEVO,False,False
2,2581019,FRIEDMAN,MATELUNA,DANIEL FERNANDO,1708790819,1993/12/18,DANIEL.FRIEDMAN@GMAIL.COM,0968521452,2,CALLE 6 N 52156 Y CALLE 11,...,23,True,650,ORELLANA ROLDAN DIEGO FERNANDO,6,0601552490,63,MEDICO NUEVO,False,False
3,2581021,MATAMOROS,ORELLANA,KAREN LIZETTE,0704012392,1966/05/24,KAREN.MATAMOROS@GMAIL.COM,0995555111,4,AVENIDA 7 N 9876 Y AVENIDA 65,...,23,True,2861,ORELLANA SALAN MARIA ELIZA,32,1600204620,59,MEDICO NUEVO,False,False
4,2581023,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,1981/10/09,MARIA.MIRANDA@GMAIL.COM,0963124451,2,AVENIDA 11 N 6546 Y CALLE 5,...,23,True,268,TITUANA SARI MARIA DE LOURDES,26,0702843731,70,MEDICO NUEVO,False,False
5,2581025,PILLAJO,BALLADARES,GLORIA JANETH,1712510419,1973/12/06,GLORIA.PILLAJO@GMAIL.COM,0991125244,4,AVENIDA 16 OE 546 Y CALLE 852,...,23,True,901,BALSECA CARRERA MARCIA JANETH,28,0501817555,62,MEDICO NUEVO,False,False
6,2581024,PEZ,SANCHEZ,MARTHA CECILIA,1703898567,1992/09/18,MARTHA.P@EZ@GMAIL.COM,0987444112,3,AVENIDA 14 S 85541,...,12,True,2799,PAZOS SARMIENTO MARTHA CECILIA,34,1707178875,75,MEDICO NUEVO,False,False
7,2581016,BRAVO,QUIJANO,RITA ANNABEL,1307604478,1995/02/14,RITA.BRAVO@GMAIL.COM,0985214566,3,CALLE 3 N 1243 Y CALLE 7,...,15,True,2467,BRAVO GARCIA ZITA ARACELI,30,1303429920,67,MEDICO NUEVO,False,False
8,2581028,MORALES,PEREZ,PIEDAD EMMA,0501600373,1962/11/01,EMMAPIEDAD.MORALESPEREZ@HOTMAIL.COM,0987126890,2,CALLE 4842 NRO 5655 Y CALLE 3866,...,5,True,47,MORALES PEREZ EMMA PIEDAD,5,0501600373,100,MEDICO EN BASE,True,True
9,2581030,CASTILLO,MORAN,IVETTE CECIBEL,0908333602,1990/12/17,IVETTECECIBEL.MORANCASTILLO@HOTMAIL.COM,0977440964,3,CALLE 4819 NRO 6809 Y CALLE 2730,...,28,True,130,MORAN CASTILLO IVETTE CECIBEL,28,0908333602,100,MEDICO EN BASE,True,True


CONEXIÓN CON MONGODB - ALMACENAMIENTO HISTORICOS

In [67]:
# Conexión al servidor MongoDB local
client = MongoClient("localhost", 27017)

# Accede a una base de datos
db = client.laboratorio

# Convierte el DataFrame en un diccionario
registros_historicos = coincidencias_df.to_dict(orient='records')

# Obtén la fecha actual
fecha_hoy = datetime.now()

# Agregar la fecha de hoy a cada registro en el diccionario
for registro in registros_historicos:
    registro['fecha_de_ejecucion'] = fecha_hoy

# Colección en la que se almacenarán los registros históricos
historico_collection = db.historicos

# Inserta los registros en la colección de historial
historico_collection.insert_many(registros_historicos)


CONEXIÓN BBD SQL SERVER PARA INSERCIÓN DE NUEVOS MEDICOS

In [68]:
# Filtrar información de médicos nuevos
carga_base = coincidencias_df[coincidencias_df['OBSERVACION'] == 'MEDICO NUEVO']

# Seleccionamos las columnas que se van a cargar
carga_base1 = carga_base.iloc[:, :12].copy()
carga_base1.head()

,CODCUP,APELLIDO_PAT,APELLIDO_MAT,NOMBRES,CI,FEC_NAC,EMAIL,TEL,Q_FICO,DIRECCION,NOMBRE_COMPLETO_EXC,ID_ESPECIALIDAD
0,2581015,ARCE,CAMPOVERDE,RAFAEL MARCOS,0702916859,1985/02/23,MARCOS.ARCE@GMAIL.COM,0998756321,2,AVENIDA 3 OE 9874 Y CALLE 1,ARCE CAMPOVERDE RAFAEL MARCOS,37
1,2581017,CARDENAS,DIAZ,FAUSTO OLIVERIO,1400237044,1960/10/31,FAUSTO.CARDENAS@GMAIL.COM,0963144522,4,CALLE 4 E 1452 Y CALLE 8,CARDENAS DIAZ FAUSTO OLIVERIO,42
2,2581019,FRIEDMAN,MATELUNA,DANIEL FERNANDO,1708790819,1993/12/18,DANIEL.FRIEDMAN@GMAIL.COM,0968521452,2,CALLE 6 N 52156 Y CALLE 11,FRIEDMAN MATELUNA DANIEL FERNANDO,23
3,2581021,MATAMOROS,ORELLANA,KAREN LIZETTE,0704012392,1966/05/24,KAREN.MATAMOROS@GMAIL.COM,0995555111,4,AVENIDA 7 N 9876 Y AVENIDA 65,MATAMOROS ORELLANA KAREN LIZETTE,23
4,2581023,MIRANDA,GARCES,MARIA DE LOURDES,1714038211,1981/10/09,MARIA.MIRANDA@GMAIL.COM,0963124451,2,AVENIDA 11 N 6546 Y CALLE 5,MIRANDA GARCES MARIA DE LOURDES,23


In [70]:
# Establece la cadena de conexión a tu servidor SQL Server
server = 'BFICHASIST23'
database = 'laboratorio'
username = 'sa'
password = '1719895086'

conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};TIMEOUT=30')
cursor = conn.cursor()

for index, row in carga_base1.iterrows():
    apellido_paterno = row['APELLIDO_PAT'] if not pd.isnull(row['APELLIDO_PAT']) else None
    apellido_materno = row['APELLIDO_MAT'] if not pd.isnull(row['APELLIDO_MAT']) else None
    nombres = row['NOMBRES'] if not pd.isnull(row['NOMBRES']) else None
    id_tipo_identificacion = '1'
    numero_identificacion = row['CI']

    # Comprobación de la fecha de nacimiento
    if isinstance(row['FEC_NAC'], str):
        try:
            fecha_nacimiento = datetime.strptime(row['FEC_NAC'], '%Y/%m/%d').strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error en la fila {index}: {e}")
            fecha_nacimiento = None
    else:
        fecha_nacimiento = row['FEC_NAC'].strftime('%Y-%m-%d') if not pd.isnull(row['FEC_NAC']) else None

    email = row['EMAIL'] if not pd.isnull(row['EMAIL']) else None
    celular = row['TEL'] if not pd.isnull(row['TEL']) else None
    direccion = row['DIRECCION'] if not pd.isnull(row['DIRECCION']) else None

    # Comprobación de id_especialidad
    id_especialidad = row['ID_ESPECIALIDAD']
    if not pd.isnull(id_especialidad):
        id_especialidad = int(id_especialidad) if isinstance(id_especialidad, (int, float)) else 0
    else:
        id_especialidad = None

    # Ejecuta la consulta
    cursor.execute("INSERT INTO dbo.medicos (apellido_paterno, apellido_materno, nombres, id_tipo_identificacion, numero_identificacion, fecha_nacimiento, email, celular, direccion, id_especialidad) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   (apellido_paterno, apellido_materno, nombres, id_tipo_identificacion, numero_identificacion, fecha_nacimiento, email, celular, direccion, id_especialidad))

# Confirma la transacción y cierra la conexión
conn.commit()
conn.close()
